<a href="https://colab.research.google.com/github/jmcconne100/Pandas_Notebook_Project/blob/main/Probability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prob_tools.py — quick probability analysis helpers
import numpy as np
import pandas as pd
from typing import Tuple, Iterable, Dict, Optional
from scipy import stats

# ---------------------------
# Empirical probabilities
# ---------------------------
def empirical_prob(df: pd.DataFrame, col: str, event) -> float:
    """P(X=event) from data column (event can be a value or a callable predicate)."""
    if callable(event):
        return (df[col].apply(event).mean())
    return (df[col] == event).mean()

def joint_prob(df: pd.DataFrame, A: str, B: str, a, b) -> float:
    """P(A=a and B=b)."""
    mask_a = df[A].apply(a) if callable(a) else (df[A] == a)
    mask_b = df[B].apply(b) if callable(b) else (df[B] == b)
    return (mask_a & mask_b).mean()

def conditional_prob(df: pd.DataFrame, A: str, B: str, a, b) -> float:
    """P(A=a | B=b)."""
    mask_b = df[B].apply(b) if callable(b) else (df[B] == b)
    if mask_b.sum() == 0:
        return np.nan
    mask_a = df[A].apply(a) if callable(a) else (df[A] == a)
    return (mask_a & mask_b).sum() / mask_b.sum()

# ---------------------------
# Contingency table & chi-square test
# ---------------------------
def contingency_analysis(df: pd.DataFrame, A: str, B: str) -> Dict[str, object]:
    """
    Returns: {"table": pd.DataFrame, "chi2": float, "p": float, "dof": int, "expected": np.ndarray}
    """
    table = pd.crosstab(df[A], df[B])
    chi2, p, dof, expected = stats.chi2_contingency(table, correction=False)
    return {"table": table, "chi2": chi2, "p": p, "dof": dof, "expected": expected}

# ---------------------------
# Distribution probabilities
# ---------------------------
def binomial_prob(k: int, n: int, p: float, tail: str = "pmf") -> float:
    """Binomial pmf/cdf/sf: tail in {"pmf","cdf","sf"} (sf = P(X>=k))."""
    rv = stats.binom(n=n, p=p)
    if tail == "pmf": return rv.pmf(k)
    if tail == "cdf": return rv.cdf(k)
    if tail == "sf":  return rv.sf(k-1)  # P(X >= k)
    raise ValueError("tail must be pmf|cdf|sf")

def poisson_prob(k: int, lam: float, tail: str = "pmf") -> float:
    rv = stats.poisson(mu=lam)
    if tail == "pmf": return rv.pmf(k)
    if tail == "cdf": return rv.cdf(k)
    if tail == "sf":  return rv.sf(k-1)
    raise ValueError("tail must be pmf|cdf|sf")

def normal_prob_interval(mu: float, sigma: float, a: float, b: float) -> float:
    """P(a <= X <= b) for X~N(mu, sigma^2)."""
    rv = stats.norm(loc=mu, scale=sigma)
    return rv.cdf(b) - rv.cdf(a)

# ---------------------------
# Proportion CI (Wilson) & Bootstrap CIs
# ---------------------------
def proportion_ci_wilson(successes: int, n: int, conf: float = 0.95) -> Tuple[float,float,float]:
    """
    Wilson score interval for a binomial proportion.
    Returns (p_hat, lo, hi).
    """
    if n <= 0:
        return (np.nan, np.nan, np.nan)
    p = successes / n
    z = stats.norm.ppf(0.5 + conf/2)
    denom = 1 + z**2/n
    center = (p + z**2/(2*n)) / denom
    half = z * np.sqrt((p*(1-p)/n) + (z**2/(4*n**2))) / denom
    return (p, center - half, center + half)

def bootstrap_ci(
    data: Iterable[float],
    stat: str = "mean",
    conf: float = 0.95,
    n_boot: int = 10000,
    random_state: Optional[int] = 42
) -> Tuple[float,float,float]:
    """
    Percentile bootstrap for mean or median.
    Returns (stat_value, lo, hi).
    """
    rng = np.random.default_rng(random_state)
    x = np.asarray(list(data), dtype=float)
    if x.size == 0:
        return (np.nan, np.nan, np.nan)
    f = np.mean if stat == "mean" else np.median
    theta = f(x)
    boots = np.array([f(rng.choice(x, size=x.size, replace=True)) for _ in range(n_boot)])
    alpha = (1 - conf) / 2
    lo, hi = np.quantile(boots, [alpha, 1 - alpha])
    return (theta, lo, hi)

# ---------------------------
# Bayes update (binary hypothesis) & simple Bayesian A/B
# ---------------------------
def bayes_update(prior: float, likelihood_H: float, likelihood_notH: float) -> float:
    """
    Posterior P(H|E) = prior * L(E|H) / [prior*L(E|H) + (1-prior)*L(E|~H)]
    """
    num = prior * likelihood_H
    den = num + (1 - prior) * likelihood_notH
    return num / den if den > 0 else np.nan

def ab_test_bayesian(
    conv_a: int, n_a: int,
    conv_b: int, n_b: int,
    prior_alpha: float = 1.0,
    prior_beta: float  = 1.0,
    draws: int = 100_000,
    random_state: int = 42
) -> Dict[str, float]:
    """
    Compare A vs B conversion via Beta-Binomial posteriors.
    Returns posterior means and P(B > A).
    """
    rng = np.random.default_rng(random_state)
    a_alpha = prior_alpha + conv_a
    a_beta  = prior_beta  + (n_a - conv_a)
    b_alpha = prior_alpha + conv_b
    b_beta  = prior_beta  + (n_b - conv_b)

    a_s = rng.beta(a_alpha, a_beta, size=draws)
    b_s = rng.beta(b_alpha, b_beta, size=draws)
    p_b_better = (b_s > a_s).mean()
    return {
        "post_mean_A": a_alpha / (a_alpha + a_beta),
        "post_mean_B": b_alpha / (b_alpha + b_beta),
        "p_B_greater_A": float(p_b_better)
    }

# ---------------------------
# Quick demo (run this cell to see example outputs)
# ---------------------------
if __name__ == "__main__":
    # Synthetic dataset for empirical probabilities
    rng = np.random.default_rng(0)
    df_demo = pd.DataFrame({
        "segment": rng.choice(["A","B"], size=1000, p=[0.6, 0.4]),
        "clicked": rng.choice([0,1], size=1000, p=[0.8, 0.2]),
        "spend":   rng.gamma(shape=2.0, scale=10.0, size=1000)
    })

    print("P(segment='A'):", round(empirical_prob(df_demo, "segment", "A"), 3))
    print("P(clicked=1):  ", round(empirical_prob(df_demo, "clicked", 1), 3))
    print("P(A & clicked):", round(joint_prob(df_demo, "segment", "clicked", "A", 1), 3))
    print("P(clicked|A):  ", round(conditional_prob(df_demo, "clicked", "segment", 1, "A"), 3))

    res = contingency_analysis(df_demo, "segment", "clicked")
    print("\nContingency table:\n", res["table"])
    print("chi2={:.3f}, p={:.4f}, dof={}".format(res["chi2"], res["p"], res["dof"]))

    print("\nBinomial: P(X>=7) for n=10, p=0.5 →", round(binomial_prob(7, 10, 0.5, "sf"), 4))
    print("Poisson:  P(X=3)  for λ=2 →", round(poisson_prob(3, 2.0, "pmf"), 4))
    print("Normal:   P(45≤X≤55) for N(50, 10^2) →", round(normal_prob_interval(50, 10, 45, 55), 4))

    phat, lo, hi = proportion_ci_wilson(successes=42, n=200, conf=0.95)
    print(f"\nWilson CI (p̂={phat:.3f}) → [{lo:.3f}, {hi:.3f}]")

    mean_hat, blo, bhi = bootstrap_ci(df_demo["spend"], stat="mean", conf=0.95, n_boot=5000)
    print(f"Bootstrap mean CI: {mean_hat:.2f} → [{blo:.2f}, {bhi:.2f}]")

    bayes = ab_test_bayesian(conv_a=20, n_a=200, conv_b=35, n_b=220)
    print("\nBayesian A/B:", bayes)